# Image Compression System

<img src="img1.png">

## Context & Problem
Machine Learning (ML) applications in computer vision is growing faster than ever. Although computing power of modern computers increases, so does the amount of data generated by numerous high-quality pictures. Therefore, efficient large-scale image processing is therefore a challenging task for domestic computers.

Imagine a picture size of 1920 x 1200 pixels, pretty common for modern cameras.
Each pixel is usually encoded with a red, green, and blue level. This means that a single picture represented in a matrix form would have easily 6.9 millions elements. Now imaging the number of pictures examples required to achieve high-accuracy for a Deep Learning neural network classifiers (> 10,000). This means that overall, we have to feed the learning algorithm with about **70 bilions elements!!** 

This is not very practical and would not run in a timely manner. Hopefully, although advanced ML algorithms need a lot of picture examples, they do not necessary require such high-quality image for the learning task. 
## Need & Motivation
This Image Compression System project (ICS) originates from the following need:
- to have an image pre-processing program that reduces the overall size of any image and constrain the number of usefull colors to its minimum (i.e. such as to retain the necessary level of information for the learning task).

This is motivated by the numerous applications it would have on further computer vision related projects, and the time saving once successfully implemented.


## Objective & Requirements
The **objective** is to implement various image reduction strategies in order to program an Image Compression System (ICS) based on K-means clustering algorithms.

The ICS shall meet the following requirements:
1. The ICS shall provide the user the choice to reduce the inputed images with the following strategies:
    1. Full K-Means clustering algorithm from sklearn library (FKM)
    2. Mini-batch K-Means clustering algorithm from sklearn libary (MBKM)
    3. Custom-build K-means clustering algorithm (MKM)
    4. Size-only compression (only reduce image size, no run of k-means). (SO)
2. Whaterver the compression strategy selected by the user is, the ICS shall run a Size-Only reduction on all inputed images.
3. The ICS shall take as input multiple images from the local folder "Uncompressed", process each image and save as output the compressed image in the local folder "Compressed".
4. The ICS shall return to the user the images processing progress, and the total program run time.
5. When a user selects a K-means clustering strategy (FKM, MBKM, or FKM), the ICS shall request from the user, the number of colors. This number will be applied for the whole image dataset, and represent the K clusters of the K-means algorithm.

In [19]:
# import libraries
import numpy as np
from PIL import Image
import os
import time
from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import KMeans

## K-Means clustering in a nutschell
In this project, the K-means algorithm is used in order to reduce the maximum number of colors use to encode the picture. Typical pictures are encoded by 255 levels of red, green, and blue, which make more than 16 millions colors possible. A reduce set of colors, allows to compress more efficiently a picture, because it requires a lower number of bits to be encoded.

let's imaging we have a 100 pixels image whose x, and y encode Red and Green levels (we omit Blue level for ease of representation in 2D). There are roughly 100 colors to store.
<img src="img2.png">



A k-means algorithm is:
1. randomly select k pre-defined centroids. (in this project, k is the number of colors)

Then iterate:
2. Given the centroids of the futur k-clusters, assign the data point to the closest centroid.
3. Given data points assignement, update the centroid position using the mean.

**Example**

Running a k-means algorithm trying to keep only 4 colors (k=4), would cluster the pixels as follow (cluster centroids are marked in red):
1. highly green, low red
2. highly green, highly red
3. moderate green, highly red
4. low green, moderate red.
<img src="img3.png">

The algorithm of K-means clustering works as follow

## Custom K-Means algorithm
In this section, we build our own functions for a custom K-means algorithm.

In [20]:
def initialize_K_centroids(X,K):
    """ We choose K points from X at random
    INPUTS:
        - X: (m x n) matrix. Each row correspond to an example (pixel), each column to R,G or B
        - K: (scalar). The number of centroids we wish to initialise.
    OUTPUT:
        - Return the coordinate of the randomly selected K points from K
    """
    # counts how many examples (data points) there are
    m = len(X)
    
    # Create an array of K indices in the range (0 - number of examples)
    idx = np.random.choice(m,size=K,replace=False)
    
    # Return only the randomly selected K points from X
    return X[idx]

def find_closest_centroids(X,centroids):
    """For each data point, find what is the closest centroid
    INPUTS:
        - X: (m x n) matrix. each row corresponds to an example (pixel), each column to R,G or B
        - centroids (k x n): each row correspond to a centroid and contains its coordinates
    OUTPUT:
        - c: (m x 1) vector. each row correspond to the assignment of a pixel to a cluster.
    """
    
    # Step 1: Compute distances for each point to each centroids
    
    #pre-allocation
    difference = np.zeros((centroids.shape[0],X.shape[0],X.shape[1])) 
    # compute result and store in a new dim(axis=2) 
    difference = X - centroids[:,np.newaxis]
    # pre-allocate distances matrix
    distances = np.zeros((centroids.shape[0],X.shape[0]))
    # computes distances
    distances = np.sqrt(np.sum(np.square(difference),axis=2)) 
    
    # Step2 : Assign closest cluster to each data point is vector c
    c = np.argmin(distances,axis=0)
    return c

def compute_means(X,c,K):
    """For each cluster of assigned datapoints, compute the mean to update the centroids
    INPUTS:
        - X: (m x n) matrix. Each row correspond to an example (pixel), each column to R,G or B
        - K: (scalar). The number of centroids we wish to initialise.
    OUTPUT:
        - centroids (k x n): updated position of the cluster centroids 
    """
    m,n = X.shape
    centroids = np.zeros((K,n)) # pre-allocate the centroids
    for k in range(K):    # for each cluster...
        examples = X[c==k] # we keep the data points that belongs to this cluster ...
        mean = np.mean(examples,axis=0) # ... and compute the mean.
        centroids[k]=mean # then we update the centroid position
    return centroids

def k_means(X,K,max_iters = 10):
    """
    Run custom k-means algorithm on the dataset
    INPUTS:
        - X (n x d) data points
        - K (scalar), the number of cluster
        - max_iters, number of repeated cluster assignment and centroid update
    OUTPUTS:
        - centroids of each cluster
        - c (n x 1) vector with assigned cluster to each data point
    """
    centroids = initialize_K_centroids(X,K)
    for i in range(max_iters):
        c = find_closest_centroids(X,centroids)
        centroids = compute_means(X,c,K)
    return centroids,c

## User Inputs Request
The following piece of code ensure compliance with requirements [1] and [2] It request to the user which image compression strategy should be applied to the set of uncompressed images.

In [21]:
print("\n Choose the Compression Algorithm:")
print("Type 'SO' for Size-Only compression")
print("Type 'MKM' for Custom K-Means compression")
print("Type 'FKM' for sklearn Full K-Means compression")
print("Type 'MBKM' for sklearn Mini-Batch K-Means compression")
algo = input("Type here :  ")

# The the selected algorithm is "SO" (i.e. size-only), there will not color clustering.
# it is therefore not necessary to ask to the user the number of colors after compression.
if algo != "SO":
    K = int(input("Enter the number of colors used after compression:  "))

# Choose and open the current working directory
os.chdir("/Users/patrickfleith/Desktop/2_DataScience /Projects/ICS")
folderPath = os.getcwd() # get the current working directory
originPath = folderPath + "/Uncompressed" # create path to the uncompressed data set
destinationPath = folderPath + "/Compressed" # create path to the compressed folder
os.chdir(originPath) # choose and open the folder with uncompressed images
image_list = os.listdir(originPath) # make a list of all image names
image_list.remove(".DS_Store") # remove non image name

count = 0 # count will track the number of image already procssed. Initially set to zero.
maxCount = len(image_list) # max number of image to be compressed.


 Choose the Compression Algorithm:
Type 'SO' for Size-Only compression
Type 'MKM' for Custom K-Means compression
Type 'FKM' for sklearn Full K-Means compression
Type 'MBKM' for sklearn Mini-Batch K-Means compression
Type here :  MBKM
Enter the number of colors used after compression:  16


In [22]:
ti = time.perf_counter() # store initial timer time

# For all images in the uncompressed folder
for file in image_list:
    im = Image.open(file) # open image
    image = np.asarray(im) # convert into a numpy array
    
    # Size reduction
    image = image[0:-1:5,0:-1:5,:] # reduce the image size (requirement [2]) by keeping only\
    # 1 pixel every 5 pixel in a row, and 1 pixel every 5 pixel in each columns.
    
    # If algorithm is Size-Only
    if algo == "SO":
        
        compressed_image = Image.fromarray(image)  # convert to Image object
        compressed_image.save(destinationPath+"/"+str(count)+".png") # save the image
    
    # otherwise, if algorithm is the custom K-means algorithm
    elif algo == "MKM":
        
        image = image/255 # normalise image values
        w,h,d = image.shape
        X = image.reshape((w * h, d)) # reshape the 3d array into a 2d array.
        
        start = time.perf_counter() # start the timer for processing of this image.
        colors, c = k_means(X, K, max_iters=10) # run the custom k-means function.
        
        # once k-means done, display image processing time
        print("Costum k-means operation took: ",round(time.perf_counter()-start),"s")
        
        # convert result value in type uint8
        c = np.array(c, dtype=np.uint8)
        
        # reconstruct the image into a 3d array
        X_reconstructed = np.array(colors[c, :] * 255, dtype=np.uint8).reshape((w, h, d))
        
        # convert numpy reconstructed image into an Image object
        compressed_image = Image.fromarray(X_reconstructed)
        
        # save the Image.
        compressed_image.save(destinationPath+"/"+str(count)+".png")
    
    # otherwise, if the strategy selected was full k-means with sklearn
    elif algo == "FKM":
        
        image = image/255
        w,h,d = image.shape
        X = image.reshape((w * h, d))
        
        start = time.perf_counter()
        kmeans = KMeans(n_clusters=K).fit(X) # apply KMeans() from sklearn libary
        c = kmeans.labels_ # retrieve data point assignments to clusters
        colors = kmeans.cluster_centers_ # retrive centroids coordinates
        
        # print run time for that given image
        print("Full sklearn k-means operation took: ",round(time.perf_counter()-start),"s")
        
        # reconstruct and save the image
        c = np.array(c, dtype=np.uint8)
        X_reconstructed = np.array(colors[c, :] * 255, dtype=np.uint8).reshape((w, h, d))
        compressed_image = Image.fromarray(X_reconstructed)
        
        compressed_image.save(destinationPath+"/"+str(count)+".png")
    
    # otherwise, if selected strategy was mini-batch k-means
    elif algo == "MBKM":
        
        image = image/255
        w,h,d = image.shape
        X = image.reshape((w * h, d))
        
        start = time.perf_counter()
        kmeans = MiniBatchKMeans(n_clusters=K,
                           random_state=0,
                           batch_size=100000,
                           max_iter=10).fit(X) # we run MiniBatchKMeans() from sklearn.
        c = kmeans.labels_
        colors = kmeans.cluster_centers_
        print("Mini-batch sklearn k-means operation took: ",round(time.perf_counter()-start),"s")
        
        c = np.array(c, dtype=np.uint8)
        X_reconstructed = np.array(colors[c, :] * 255, dtype=np.uint8).reshape((w, h, d))
        compressed_image = Image.fromarray(X_reconstructed)
        
        compressed_image.save(destinationPath+"/"+str(count)+".png")

    # after the current image has been processed and saved, we increment the progress counter
    count += 1
    # display progress
    print("Progress = ",round(100*count/maxCount),"%") 

# Once all images have been processed...
tf = time.perf_counter() # record final time
print("Total Runtime: ",round(tf-ti),"s") # print final runtime (Compliance to requirement [4])

Mini-batch sklearn k-means operation took:  5 s
Progress =  17 %
Mini-batch sklearn k-means operation took:  5 s
Progress =  33 %
Mini-batch sklearn k-means operation took:  2 s
Progress =  50 %
Mini-batch sklearn k-means operation took:  5 s
Progress =  67 %
Mini-batch sklearn k-means operation took:  8 s
Progress =  83 %
Mini-batch sklearn k-means operation took:  7 s
Progress =  100 %
Total Runtime:  34 s


## Results and Discussion
Let's compare the performances of the different image compression stratgies.

Performances in terms of run time and compression factor are compared in the table below when run on 5 images, with a total of 16 Mo, and for k=32 colors when clustering is used. We can see that SO strategy is very fast (2s) in comparison to the other technics (>30s). This is because we do not run any clustering algorithm. When clustering is employed, it is much slower but the compression factor achieved are almost 4 times greater. Among clustering strategies, Mini-batch k-means is the fastest of all.
<img src="img4.png">

Among cluster-based strategies, we can compare the output images. It actually appears that differencce between clustered images is negligeable. Feel free to play around with the code to figure it out. Below, we provide a couple of examples of image compressed using Mini-batch.



**ORIGINAL IMAGE**
The challenge for reducing this image is the variety of colors (many levels of red, blue, and green) that needs to be encoded in only a few (32 colors).
<img src="img5.png">

**COMPRESSED IMAGE (Mini-Batch K-means)**

<img src="img6.png">

**ORIGINAL IMAGE**
The challenge for compressing the following image is the webb which is made of very thin strips of pixels.
<img src="img7.png">

**COMPRESSED IMAGE (Mini-Batch K-means)**

<img src="img8.png">

## Futur Work and improvement:
As a futur work, we suggest the following improvement:
- Ask user the pixel-skipping step, or;
- Ask user the final image size to generate compressed images of the the same size;
- Make a notebook which describes the project;
- (optional) Add progress bar with tqdm library.